In [1]:
spark.conf.set("spark.sql.shuffle.partitions", 1000)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [2]:
from pyspark.ml.clustering import KMeans, KMeansModel, BisectingKMeans, BisectingKMeansModel
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA, PCAModel, ElementwiseProduct
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.functions import col, sum, greatest, expr, count, countDistinct, max, min, variance, bround, avg, lit, udf
from pyspark.sql import Row
import pandas as pd
import scipy.stats as stats
import matplotlib.pylab as plt
from pyspark.ml.linalg import Vectors
from pyspark.ml.linalg import SparseVector, DenseVector

#PARA EL TRATAMIENTO DE OUTLIERS
from pyspark.sql.functions import col, count, when
from functools import reduce
from operator import and_


### Data Importation

In [3]:
dfEM=spark.read.parquet("/analytics/bc1_clustering/tables/dfEM")

In [5]:
#dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfBufferAccYrGrpPart")
#dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfBufferAccYrSubPart2")
# dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfBufferAccYrGrp")
# dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfFinalMatrix11Dic")
# dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfFinalMatrixGrpVal19Dic")
dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfFinalMatrixSubVal04Ene")

# dfBufferOrig=dfBufferOrig.withColumnRenamed('','NULO')

In [6]:
# dfBufferOrig.persist()

nameList=dfBufferOrig.schema.names[1:]

len(nameList)

3187

In [ ]:
dfBufferOrig.count()

In [ ]:
# [dfBuffer, dfTestBuffer] = dfBufferOrig.randomSplit([0.5, 0.5],666)

In [7]:
dfBuffer=dfBufferOrig.where(~dfBufferOrig['ACCOUNTPK'].cast('string').substr(-1,1).isin(['0','1','2','3','4'])).persist()

In [8]:
dfTestBuffer=dfBufferOrig.where(dfBufferOrig['ACCOUNTPK'].cast('string').substr(-1,1).isin(['0','1','2','3','4'])).persist()

In [ ]:
#dfBuffer.write.parquet("/analytics/bc1_clustering/tables/dfTrainBufferBin")

In [ ]:
#dfTestBuffer.write.parquet("/analytics/bc1_clustering/tables/dfTestBufferBin")

In [ ]:
# dfBuffer=spark.read.parquet("/analytics/bc1_clustering/tables/dfTrainBufferBin")
# dfBuffer.persist()
# nameList=dfBuffer.schema.names[1:]

In [ ]:
# dfTestBuffer=spark.read.parquet("/analytics/bc1_clustering/tables/dfTestBufferBin")
# dfTestBuffer.persist()

In [9]:
dfBuffer.count()

1980853

In [8]:
dfTestBuffer.count()

2010927

In [ ]:
#dfTestBuffer.select(variance(dfTestBuffer.columns[2])).show()

In [ ]:
#dfBufferOrig.select(dfBufferOrig['ACCOUNTPK'].cast('string').substr(-1,1).alias('caracter')).groupBy('caracter').count().show()

In [ ]:
# dfTsys=spark.read.parquet("/analytics/bc1_clustering/tables/dfTsysAccYrOriginal")

In [ ]:
# dfTsys.persist()

In [10]:
dfTsys=dfBufferOrig.select('ACCOUNTPK','Emprestimo_Pessoal','Pagamento_de_Contas','Parcelamento_Fatura_Automatico','Parcelamento_Fatura_Facil','Saque_Parcelado','Saque_a_Vista','Overlimit')

### Binary matrix creation

In [ ]:
dfBufferBinary=dfBufferOrig.select([ i if i=='ACCOUNTPK' else when(col(i)!=0,1).otherwise(0).alias(i) for i in dfBufferOrig.columns])

In [ ]:
dfBufferBinary.persist().count()

In [ ]:
dfBufferBinary.write.parquet("/analytics/bc1_clustering/tables/dfBufferAccYrSubPartBin2")

In [ ]:
# #TEST CONVERSOR A MATRIZ 0/1
# l = [('A',2.0,-3.0,0,-30.0),('B',0,-300.0,1,0),('C',7.0,0.0,2,-50.0),('D',12.0,-700.0,0,0)]
# rdd = sc.parallelize(l)
# dummy = rdd.map(lambda x: Row(ID=x[0], col1=float(x[1]),col2=float(x[2]),col3=float(x[3]),col4=float(x[4])))
# dummyDf = sqlContext.createDataFrame(dummy)
# dummyDf.show(5)
# dummyDf.select([ i if i=='ID' else when(col(i)!=0,1).otherwise(0).alias(i) for i in dummyDf.columns]).show()

### Variable distribution histogram

In [ ]:
#METODO HISTOGRAM DF
plt.clf()
#bins, counts = dummyDf.select('col1').rdd.flatMap(lambda x: x).histogram(20)
bins, counts = dfBuffer.select('CREDITLIMIT').rdd.flatMap(lambda x: x).histogram(20)
plt.hist(bins[:-1], bins=bins, weights=counts)

plt.show()
#.filter(col('101054')>0)

In [ ]:
# #METODO hist DE PLT, parece que no funciona con grandes cantidades de datos
# plt.clf()
# #n, bins, patches = plt.hist(dummyDf.select('col4').collect(), 15, normed=1, facecolor='green', alpha=0.75)
# n, bins, patches = plt.hist(dfBuffer.select('1010540800').collect(), 15, normed=1, facecolor='green', alpha=0.75)
# plt.show()

### Outlier treatment (for money value matrices)

In [ ]:
len(nameList)

In [ ]:
listaCols=[]
listaLimit=[]

for i in nameList:
    cuenta=dfBuffer.filter(col(i)>0).count()
    if cuenta>0:
        listaCols.append(i)
        Q1, Q3 = dfBuffer.filter(col(i)>0).approxQuantile(col=i,probabilities=[0.25,0.75],relativeError=0.01)
        IQR=(Q3-Q1)
        top_limit=Q3+9*IQR
        listaLimit.append(top_limit)

In [ ]:
expr=reduce(and_, [col(c) < t for c, t in zip(listaCols, listaLimit)])

In [ ]:
dfBufferOutlier=dfBuffer.where(expr)

In [ ]:
dfBufferOutlier.persist().count()

In [ ]:
dfBufferOutlier.write.parquet("/analytics/bc1_clustering/tables/dfBufferOutlier")

In [ ]:
dfBufferOutlier=spark.read.parquet("/analytics/bc1_clustering/tables/dfBufferOutlier")

In [ ]:
dfBufferOutlier.count()

### Feature Assembler and Scaler

#### Binary matrices

In [10]:
dfBufferOutlier=dfBuffer
#dfBufferOutlier=dfTestBuffer.select(dfTestBuffer.columns[0:2400])

In [11]:
#continuous_feature_assembler= VectorAssembler(inputCols=dfTestBuffer.columns[0:2400], outputCol="features")
continuous_feature_assembler= VectorAssembler(inputCols=nameList, outputCol="features")

In [12]:
estimatorsKmeans = [continuous_feature_assembler]

featurePipeline = Pipeline(stages=estimatorsKmeans)

sparkFeaturePipelineModel = featurePipeline.fit(dfBufferOutlier)

In [13]:
dfBufferFeature=sparkFeaturePipelineModel.transform(dfBufferOutlier)
#.persist()

In [17]:
dfBufferFeature.count()

KeyboardInterrupt: 

#### Money value matrices

In [11]:
#SI NO SE EJECUTA EL PROCESO DE OUTLIERS
dfBufferOutlier=dfBuffer
#dfBufferOutlier=dfTestBuffer.select(dfTestBuffer.columns[0:2400])

In [12]:
nameList=dfBuffer.schema.names[1:]

In [13]:
continuous_feature_assembler= VectorAssembler(inputCols=nameList, outputCol="unscaled_continuous_features")

In [14]:
continuous_feature_scaler = StandardScaler(inputCol="unscaled_continuous_features", outputCol="scaled_features",\
                                           withStd=True, withMean=True)

In [15]:
estimatorsKmeans = [continuous_feature_assembler, continuous_feature_scaler]

featurePipeline = Pipeline(stages=estimatorsKmeans)

sparkFeaturePipelineModel = featurePipeline.fit(dfBufferOutlier)

In [16]:
dfBufferScaled=sparkFeaturePipelineModel.transform(dfBufferOutlier)
# .persist()

In [17]:
dfBufferScaled.count()

1980853

In [20]:
dfBufferScaled.printSchema()

root
 |-- ACCOUNTPK: long (nullable = true)
 |-- BALANCE: double (nullable = true)
 |-- INSTALMENTBALANCEFOROVLMCHECK: double (nullable = true)
 |-- CREDITLIMIT: double (nullable = true)
 |-- LimitUse: double (nullable = true)
 |-- LimitOffUs: double (nullable = true)
 |-- LimitPersLoan: double (nullable = true)
 |-- LimitCashOut: double (nullable = true)
 |-- CLOSING_BALANCE: double (nullable = true)
 |-- INSTALLMENTSOFFERS: long (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- Pagamento_de_Contas: long (nullable = true)
 |-- Parcelamento_Fatura_Automatico: long (nullable = true)
 |-- Parcelamento_Fatura_Facil: long (nullable = true)
 |-- Saque_Parcelado: long (nullable = true)
 |-- Saque_a_Vista: long (nullable = true)
 |-- Overlimit: long (nullable = true)
 |-- Overlimit_Auth: integer (nullable = true)
 |-- NULO: double (nullable = true)
 |-- 0742: double (nullable = true)
 |-- 0763: double (nullable = true)
 |-- 0780: double (nullable = true)
 |-- 1520: double

#### Increase of the imporance of the financial variables (money)

In [18]:
listaPesos=([1]*3187)
#listaPesos[0:17]=[10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10]
VectorPesos=DenseVector(listaPesos)

In [19]:
transformer = ElementwiseProduct(scalingVec=VectorPesos,
                                 inputCol="scaled_features", outputCol="features")
# Batch transform the vectors to create new column:
dfBufferFeature=transformer.transform(dfBufferScaled)

In [34]:
dfBufferFeature.select('scaled_features','features').show(1,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Dimensionality reduction (WIP)

In [ ]:
pca = PCA(k=120, inputCol="features", outputCol="pca_features")

In [ ]:
pcamodel = pca.fit(dfBufferFeature)

In [ ]:
variance=pcamodel.explainedVariance

In [ ]:
variance

In [ ]:
variance.sum()

In [ ]:
dfBufferFeaturePCA=pcamodel.transform(dfBufferFeature)

### K Evaluation

In [ ]:
listaErrores=[]

for k in range(2,51):
    #print(k)
    kmeans = KMeans().setK(k).setSeed(666)
    #kmeans = BisectingKMeans().setK(k).setSeed(666)
    model = kmeans.fit(dfBufferFeature)
    wssse = model.computeCost(dfBufferFeature)
    print("Within Set Sum of Squared Errors {} = ".format(k) + str(wssse))
    listaErrores.append((k,wssse))

In [ ]:
pdErrors=pd.DataFrame.from_records(listaErrores, columns=['clusters','wssse'])

In [ ]:
plt.clf()
plt.plot(pdErrors.clusters,pdErrors.wssse)
plt.show()

In [ ]:
dfErrors=spark.createDataFrame(pdErrors)

In [ ]:
dfErrors.printSchema()

In [ ]:
# dfErrors.write.parquet("/analytics/bc1_clustering/tables/dfErrorsBin")

### Segmentation final model

In [23]:
#kmeans = KMeans().setK(16).setSeed(666)
kmeans = BisectingKMeans().setK(20).setMinDivisibleClusterSize(40000).setSeed(666)

In [36]:
dfBufferFeature.select("ACCOUNTPK","features").show(5)

+---------+--------------------+
|ACCOUNTPK|            features|
+---------+--------------------+
|    67805|[0.53609270966588...|
|   109366|[0.53609270966588...|
|   140126|[-3.4885453062219...|
|   142317|[-0.1599475963050...|
|   164347|[0.53609270966588...|
+---------+--------------------+
only showing top 5 rows



In [26]:
dfCaluculoSub=dfBufferFeature.select("ACCOUNTPK","features")

In [27]:
dfCaluculoSub.persist().count()

1980853

In [28]:
# model = kmeans.fit(dfBufferFeature)
model = kmeans.fit(dfCaluculoSub)

In [50]:
dfCaluculoSub.unpersist()

DataFrame[ACCOUNTPK: bigint, features: vector]

### Financial Product Inclination

#### Train

In [22]:
dfTrainRes=model.transform(dfBufferFeature).select(col("ACCOUNTPK").alias("ACCOUNT"),"prediction")

In [23]:
dfProdIncl=dfTrainRes.join(dfBuffer,dfTrainRes.ACCOUNT==dfTsys.ACCOUNTPK,'left_outer').fillna(0)

In [24]:
dfProdIncl.printSchema()

root
 |-- ACCOUNT: long (nullable = true)
 |-- prediction: integer (nullable = true)
 |-- ACCOUNTPK: long (nullable = true)
 |-- BALANCE: double (nullable = false)
 |-- INSTALMENTBALANCEFOROVLMCHECK: double (nullable = false)
 |-- CREDITLIMIT: double (nullable = false)
 |-- LimitUse: double (nullable = false)
 |-- LimitOffUs: double (nullable = false)
 |-- LimitPersLoan: double (nullable = false)
 |-- LimitCashOut: double (nullable = false)
 |-- CLOSING_BALANCE: double (nullable = false)
 |-- INSTALLMENTSOFFERS: long (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- Pagamento_de_Contas: long (nullable = true)
 |-- Parcelamento_Fatura_Automatico: long (nullable = true)
 |-- Parcelamento_Fatura_Facil: long (nullable = true)
 |-- Saque_Parcelado: long (nullable = true)
 |-- Saque_a_Vista: long (nullable = true)
 |-- Overlimit: long (nullable = true)
 |-- Overlimit_Auth: integer (nullable = true)
 |-- NULO: double (nullable = false)
 |-- 0742: double (nullable = false)

In [33]:
dfProdIncl.createOrReplaceTempView("tProdIncl")

In [34]:
dfClusIncl=spark.sql("""SELECT prediction, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) Overlimit
    from tProdIncl group by prediction order by prediction""")

In [35]:
dfClusIncl.toPandas()

,prediction,Emprestimo_Pessoal,Pagamento_de_Contas,Parcelamento_Fatura_Automatico,Parcelamento_Fatura_Facil,Saque_Parcelado,Saque_a_Vista,Overlimit
0,0,0.002688,0.010415,0.115723,0.127491,0.002976,0.003536,0.156503
1,1,0.000211,0.008498,0.025100,0.071618,0.001351,0.001962,0.113970
2,2,0.001523,0.010145,0.022083,0.104396,0.006468,0.008660,0.062361
3,3,0.007516,0.017857,0.010084,0.096846,0.016876,0.009337,0.067645
4,4,0.002490,0.015965,0.064108,0.097747,0.003738,0.005122,0.106466
5,5,0.001646,0.013838,0.079043,0.103799,0.003106,0.003593,0.159222
6,6,0.000098,0.059268,0.273760,0.316433,0.000000,0.000000,0.457293
7,7,0.121359,0.126453,0.275478,0.491317,0.470913,0.592947,0.420928
8,8,0.008576,0.024811,0.059303,0.137990,0.023190,0.019760,0.214986
9,9,0.011705,0.030437,0.046265,0.123025,0.022006,0.014099,0.142651


In [36]:
dfTrainRes.select("ACCOUNT","prediction").groupBy("prediction").count().orderBy("prediction").toPandas()

,prediction,count
0,0,458363
1,1,132509
2,2,374947
3,3,42841
4,4,169059
5,5,43141
6,6,225621
7,7,35531
8,8,42561
9,9,70571


In [37]:
dfImportanceCluster=spark.sql("""
    SELECT prediction, 
    Emprestimo_Pessoal/AVG_Emprestimo_Pessoal AS Emprestimo_Pessoal, 
    Pagamento_de_Contas/AVG_Pagamento_de_Contas AS Pagamento_de_Contas,
    Parcelamento_Fatura_Automatico/AVG_Parcelamento_Fatura_Automatico AS Parcelamento_Fatura_Automatico,
    Parcelamento_Fatura_Facil/AVG_Parcelamento_Fatura_Facil AS Parcelamento_Fatura_Facil,
    Saque_Parcelado/AVG_Saque_Parcelado  AS Saque_Parcelado,
    Saque_a_Vista/AVG_Saque_a_Vista  AS Saque_a_Vista,
    Overlimit/AVG_Overlimit  AS Overlimit
    FROM
    (SELECT prediction, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) Overlimit
    from tProdIncl group by prediction) CLUSTER CROSS JOIN
    (SELECT  
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Overlimit
    from tProdIncl)
    WHERE 
    order by prediction""")

In [38]:
dfImportanceCluster.columns[1:]

['Emprestimo_Pessoal',
 'Pagamento_de_Contas',
 'Parcelamento_Fatura_Automatico',
 'Parcelamento_Fatura_Facil',
 'Saque_Parcelado',
 'Saque_a_Vista',
 'Overlimit']

In [39]:
cond = "psf.when" + ".when".join(["(psf.col('" + c + "') == psf.col('max_value'), psf.lit('" + c + "'))" for c in dfImportanceCluster.columns[1:]])
import pyspark.sql.functions as psf
dfImportanceCluster.withColumn("max_value", psf.greatest(*dfImportanceCluster.columns[1:]))\
    .withColumn("MAX", eval(cond))\
    .toPandas()

,prediction,Emprestimo_Pessoal,Pagamento_de_Contas,Parcelamento_Fatura_Automatico,Parcelamento_Fatura_Facil,Saque_Parcelado,Saque_a_Vista,Overlimit,max_value,MAX
0,0,0.428610,0.346930,1.108498,0.849600,0.161905,0.159178,0.814126,1.108498,Parcelamento_Fatura_Automatico
1,1,0.033696,0.283049,0.240433,0.477262,0.073496,0.088316,0.592870,0.592870,Overlimit
2,2,0.242844,0.337940,0.211532,0.695698,0.351882,0.389783,0.324401,0.695698,Parcelamento_Fatura_Facil
3,3,1.198552,0.594800,0.096592,0.645387,0.918193,0.420253,0.351892,1.198552,Emprestimo_Pessoal
4,4,0.397104,0.531782,0.614083,0.651387,0.203392,0.230563,0.553835,0.651387,Parcelamento_Fatura_Facil
5,5,0.262439,0.460949,0.757148,0.691720,0.168994,0.161716,0.828273,0.828273,Overlimit
6,6,0.015549,1.974176,2.622324,2.108718,0.000000,0.000000,2.378839,2.622324,Parcelamento_Fatura_Automatico
7,7,19.352276,4.212093,2.638779,3.274149,25.620987,26.688651,2.189667,26.688651,Saque_a_Vista
8,8,1.367545,0.826458,0.568060,0.919569,1.261714,0.889395,1.118354,1.367545,Emprestimo_Pessoal
9,9,1.866442,1.013858,0.443173,0.819841,1.197293,0.634611,0.742068,1.866442,Emprestimo_Pessoal


In [40]:
#dfTrainRes.write.parquet("/analytics/bc1_clustering/tables/dfTrainRes")

In [41]:
#dfTrainRes=spark.read.parquet("/analytics/bc1_clustering/tables/dfTrainRes")
#dfTrainRes.persist()

#### Test

In [25]:
dfTestBufferFeatureTmp=sparkFeaturePipelineModel.transform(dfTestBuffer).persist()

In [26]:
dfTestBufferFeature=transformer.transform(dfTestBufferFeatureTmp)

In [27]:
dfTestRes=model.transform(dfTestBufferFeature).select(col("ACCOUNTPK").alias("ACCOUNT"),"prediction")

In [28]:
dfTestProdIncl=dfTestRes.join(dfTsys,dfTestRes.ACCOUNT==dfTsys.ACCOUNTPK,'left_outer').fillna(0)

In [46]:
dfTestProdIncl.printSchema()

root
 |-- ACCOUNT: long (nullable = true)
 |-- prediction: integer (nullable = true)
 |-- ACCOUNTPK: long (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- Pagamento_de_Contas: long (nullable = true)
 |-- Parcelamento_Fatura_Automatico: long (nullable = true)
 |-- Parcelamento_Fatura_Facil: long (nullable = true)
 |-- Saque_Parcelado: long (nullable = true)
 |-- Saque_a_Vista: long (nullable = true)
 |-- Overlimit: long (nullable = true)



In [47]:
dfTestProdIncl.createOrReplaceTempView("tTestProdIncl")

In [48]:
dfTestClusIncl=spark.sql("""SELECT prediction, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) Overlimit
    from tTestProdIncl group by prediction order by prediction""")

In [51]:
dfTestClusIncl.toPandas()

,prediction,Emprestimo_Pessoal,Pagamento_de_Contas,Parcelamento_Fatura_Automatico,Parcelamento_Fatura_Facil,Saque_Parcelado,Saque_a_Vista,Overlimit
0,0,0.002868,0.010116,0.116346,0.128591,0.003108,0.003582,0.154930
1,1,0.000256,0.008348,0.025331,0.072245,0.001370,0.001980,0.114597
2,2,0.001563,0.010129,0.022197,0.105862,0.006402,0.009164,0.062236
3,3,0.007571,0.017308,0.009668,0.098959,0.015934,0.009039,0.065111
4,4,0.002514,0.016198,0.065523,0.098210,0.004102,0.004832,0.107943
5,5,0.001685,0.013058,0.081762,0.102190,0.003112,0.003416,0.162494
6,6,0.000078,0.059656,0.273927,0.316287,0.000000,0.000004,0.450253
7,7,0.124010,0.129744,0.280921,0.488724,0.469912,0.589882,0.410091
8,8,0.007643,0.023093,0.059962,0.137905,0.021536,0.019422,0.212480
9,9,0.011472,0.028539,0.047124,0.123085,0.023882,0.015235,0.140435


In [52]:
dfTestRes.select("ACCOUNT","prediction").groupBy("prediction").count().orderBy("prediction").toPandas()

,prediction,count
0,0,464023
1,1,132839
2,2,383347
3,3,42927
4,4,172610
5,5,42734
6,6,230857
7,7,36626
8,8,43044
9,9,70431


#### Total

In [29]:
dfTotal=dfTrainRes.union(dfTestRes)

In [30]:
dfTotalProdIncl=dfTotal.join(dfTsys,dfTrainRes.ACCOUNT==dfTsys.ACCOUNTPK,'left_outer').fillna(0)

In [31]:
dfTotalProdIncl.printSchema()

root
 |-- ACCOUNT: long (nullable = true)
 |-- prediction: integer (nullable = true)
 |-- ACCOUNTPK: long (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- Pagamento_de_Contas: long (nullable = true)
 |-- Parcelamento_Fatura_Automatico: long (nullable = true)
 |-- Parcelamento_Fatura_Facil: long (nullable = true)
 |-- Saque_Parcelado: long (nullable = true)
 |-- Saque_a_Vista: long (nullable = true)
 |-- Overlimit: long (nullable = true)



In [32]:
dfTotalProdIncl.createOrReplaceTempView("tTotalProdIncl")

In [33]:
dfTotalClusIncl=spark.sql("""SELECT prediction, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) Overlimit
    from tTotalProdIncl group by prediction order by prediction""")

In [58]:
dfTotalClusIncl.toPandas()

,prediction,Emprestimo_Pessoal,Pagamento_de_Contas,Parcelamento_Fatura_Automatico,Parcelamento_Fatura_Facil,Saque_Parcelado,Saque_a_Vista,Overlimit
0,0,0.002779,0.010265,0.116036,0.128044,0.003042,0.003559,0.155711
1,1,0.000234,0.008423,0.025216,0.071932,0.001360,0.001971,0.114284
2,2,0.001543,0.010137,0.022140,0.105137,0.006434,0.008915,0.062298
3,3,0.007544,0.017582,0.009875,0.097904,0.016405,0.009188,0.066377
4,4,0.002502,0.016083,0.064823,0.097981,0.003922,0.004976,0.107212
5,5,0.001665,0.013450,0.080396,0.102999,0.003109,0.003505,0.160850
6,6,0.000088,0.059464,0.273845,0.316359,0.000000,0.000002,0.453733
7,7,0.122705,0.128123,0.278241,0.490001,0.470405,0.591391,0.415427
8,8,0.008107,0.023947,0.059634,0.137948,0.022359,0.019590,0.213726
9,9,0.011588,0.029489,0.046694,0.123055,0.022943,0.014666,0.141544


In [34]:
dfTotalClusIncl.createOrReplaceTempView("tTotalClusIncl")

In [35]:
dfOrden=spark.sql("""SELECT prediction,Emprestimo_Pessoal, 
    row_number() over (order by Emprestimo_Pessoal desc) as orden
    from  tTotalClusIncl order by Emprestimo_Pessoal desc""")

In [36]:
dfOrden.toPandas()

,prediction,Emprestimo_Pessoal,orden
0,7,0.122705,1
1,10,0.019999,2
2,11,0.018106,3
3,12,0.017437,4
4,9,0.011588,5
5,15,0.009835,6
6,18,0.009778,7
7,16,0.009638,8
8,19,0.009552,9
9,13,0.009372,10


In [62]:
dfTotal.select("ACCOUNT","prediction").groupBy("prediction").count().orderBy("prediction") \
    .withColumn('Percentage', col('count')/dfTotal.count()).toPandas()

,prediction,count,Percentage
0,0,922386,0.231071
1,1,265348,0.066474
2,2,758294,0.189964
3,3,85768,0.021486
4,4,341669,0.085593
5,5,85875,0.021513
6,6,456478,0.114354
7,7,72157,0.018076
8,8,85605,0.021445
9,9,141002,0.035323


In [63]:
dfTotalImportanceCluster=spark.sql("""
    SELECT prediction, 
    Emprestimo_Pessoal/AVG_Emprestimo_Pessoal AS Emprestimo_Pessoal, 
    Pagamento_de_Contas/AVG_Pagamento_de_Contas AS Pagamento_de_Contas,
    Parcelamento_Fatura_Automatico/AVG_Parcelamento_Fatura_Automatico AS Parcelamento_Fatura_Automatico,
    Parcelamento_Fatura_Facil/AVG_Parcelamento_Fatura_Facil AS Parcelamento_Fatura_Facil,
    Saque_Parcelado/AVG_Saque_Parcelado  AS Saque_Parcelado,
    Saque_a_Vista/AVG_Saque_a_Vista  AS Saque_a_Vista,
    Overlimit/AVG_Overlimit  AS Overlimit
    FROM
    (SELECT prediction, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) Overlimit
    from tTotalProdIncl group by prediction) CLUSTER CROSS JOIN
    (SELECT  
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Overlimit
    from tTotalProdIncl)
    WHERE 
    order by prediction""")

In [64]:
dfOverallPop=spark.sql("""SELECT  
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Overlimit
    from tTotalProdIncl
    """)

In [65]:
dfOverallPop.toPandas()

,AVG_Emprestimo_Pessoal,AVG_Pagamento_de_Contas,AVG_Parcelamento_Fatura_Automatico,AVG_Parcelamento_Fatura_Facil,AVG_Saque_Parcelado,AVG_Saque_a_Vista,AVG_Overlimit
0,0.006323,0.030089,0.104702,0.150508,0.018516,0.022339,0.191878


In [66]:
dfTotalImportanceCluster.columns[1:]

['Emprestimo_Pessoal',
 'Pagamento_de_Contas',
 'Parcelamento_Fatura_Automatico',
 'Parcelamento_Fatura_Facil',
 'Saque_Parcelado',
 'Saque_a_Vista',
 'Overlimit']

In [67]:
cond = "psf.when" + ".when".join(["(psf.col('" + c + "') == psf.col('max_value'), psf.lit('" + c + "'))" for c in dfTotalImportanceCluster.columns[1:]])
import pyspark.sql.functions as psf
dfTotalImportanceCluster.withColumn("max_value", psf.greatest(*dfTotalImportanceCluster.columns[1:]))\
    .withColumn("MAX", eval(cond))\
    .toPandas()

,prediction,Emprestimo_Pessoal,Pagamento_de_Contas,Parcelamento_Fatura_Automatico,Parcelamento_Fatura_Facil,Saque_Parcelado,Saque_a_Vista,Overlimit,max_value,MAX
0,0,0.439419,0.341140,1.108249,0.850747,0.164296,0.159328,0.811514,1.108249,Parcelamento_Fatura_Automatico
1,1,0.036950,0.279930,0.240835,0.477928,0.073476,0.088231,0.595608,0.595608,Overlimit
2,2,0.244001,0.336904,0.211462,0.698551,0.347492,0.399064,0.324674,0.698551,Parcelamento_Fatura_Facil
3,3,1.192949,0.584337,0.094320,0.650489,0.885973,0.411277,0.345933,1.192949,Emprestimo_Pessoal
4,4,0.395734,0.534502,0.619118,0.651001,0.211812,0.222729,0.558752,0.651001,Parcelamento_Fatura_Facil
5,5,0.263337,0.446995,0.767854,0.684340,0.167918,0.156904,0.838295,0.838295,Overlimit
6,6,0.013857,1.976248,2.615462,2.101945,0.000000,0.000098,2.364699,2.615462,Parcelamento_Fatura_Automatico
7,7,19.404564,4.258100,2.657448,3.255652,25.405246,26.473291,2.165065,26.473291,Saque_a_Vista
8,8,1.282045,0.795870,0.569562,0.916548,1.207520,0.876934,1.113866,1.282045,Emprestimo_Pessoal
9,9,1.832608,0.980046,0.445973,0.817599,1.239084,0.656536,0.737679,1.832608,Emprestimo_Pessoal


### Cluster Analysis

#### KS2 Evaluation

In [37]:
dfOrden.persist().toPandas()

,prediction,Emprestimo_Pessoal,orden
0,7,0.122705,1
1,10,0.019999,2
2,11,0.018106,3
3,12,0.017437,4
4,9,0.011588,5
5,15,0.009835,6
6,18,0.009778,7
7,16,0.009638,8
8,19,0.009552,9
9,13,0.009372,10


In [38]:
dfValidacionNov=spark.read.parquet("/analytics/bc1_clustering/tables/dfValidacionNov")

In [39]:
dfValidacionNov.printSchema()

root
 |-- TRANSACTIONSERNO: long (nullable = true)
 |-- ACCOUNTSERNOFK: long (nullable = true)
 |-- TYPESERNO_GLEDGER: long (nullable = true)
 |-- VALUEDATE: string (nullable = true)
 |-- I013_TRXN_DATE: string (nullable = true)
 |-- POSTDATE: string (nullable = true)
 |-- STGENERAL: string (nullable = true)
 |-- REFDATE: timestamp (nullable = true)
 |-- FG_CURRENT: integer (nullable = true)
 |-- TM_INSERT: timestamp (nullable = true)
 |-- PRODUTO: string (nullable = true)



In [40]:
dfValidacionNov.where(col('PRODUTO').like('Empr%')).select('ACCOUNTSERNOFK')

DataFrame[ACCOUNTSERNOFK: bigint]

In [41]:
dfEvalTmp1=dfTotalProdIncl.select('ACCOUNT','prediction','Emprestimo_Pessoal') \
    .join(dfValidacionNov.where(col('PRODUTO').like('Empr%')).select('ACCOUNTSERNOFK'),dfValidacionNov.ACCOUNTSERNOFK==dfTotalProdIncl.ACCOUNT,'left_outer') \
    .withColumn('EP',when(col('ACCOUNTSERNOFK').isNull(),0).otherwise(1)) 

In [42]:
dfEvalTmp1.persist()

DataFrame[ACCOUNT: bigint, prediction: int, Emprestimo_Pessoal: bigint, ACCOUNTSERNOFK: bigint, EP: int]

In [44]:
dfEvalTmp1.groupBy('EP','Emprestimo_Pessoal').count().show()

+---+------------------+-------+
| EP|Emprestimo_Pessoal|  count|
+---+------------------+-------+
|  1|                 1|     23|
|  0|                 0|3965535|
|  1|                 0|   1003|
|  0|                 1|  25219|
+---+------------------+-------+



In [45]:
dfEvalTmp1.printSchema()

root
 |-- ACCOUNT: long (nullable = true)
 |-- prediction: integer (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- ACCOUNTSERNOFK: long (nullable = true)
 |-- EP: integer (nullable = false)



In [76]:
# dfEval=dfEvalTmp1.join(dfOrden,dfEvalTmp1.prediction==dfOrden.prediction).drop(dfOrden.prediction).drop(dfOrden.Emprestimo_Pessoal) \
#     .drop(dfEvalTmp1.ACCOUNTSERNOFK)

In [43]:
dfEval=dfEvalTmp1.join(dfOrden,dfEvalTmp1.prediction==dfOrden.prediction) \
    .select('ACCOUNT',dfEvalTmp1.prediction, 'EP','orden')

In [47]:
dfEval.printSchema()

root
 |-- ACCOUNT: long (nullable = true)
 |-- prediction: integer (nullable = true)
 |-- EP: integer (nullable = false)
 |-- orden: integer (nullable = true)



In [51]:
Positives=[i.orden for i in dfEval.where(col('EP')==1).select('orden').collect()]

In [52]:
Negatives=[i.orden for i in dfEval.where(col('EP')==0).select('orden').collect()]

In [53]:
stats.ks_2samp(Positives,Negatives)

Ks_2sampResult(statistic=0.37217499106885371, pvalue=8.7024367217743854e-125)

In [56]:
dfEval.groupBy('prediction').agg((sum('EP')/count('*')).alias('porcentaje'),sum('EP'),count('*')).orderBy('porcentaje',ascending=False).toPandas()

,prediction,porcentaje,sum(EP),count(1)
0,12,0.001451,47,32402
1,10,0.001370,45,32852
2,11,0.000905,96,106043
3,8,0.000806,69,85605
4,9,0.000723,102,141002
5,7,0.000651,47,72157
6,19,0.000631,15,23765
7,15,0.000586,45,76765
8,13,0.000451,99,219596
9,17,0.000442,16,36206


#### Variable analysis

In [ ]:
dfAnVar=dfBufferOrig.join(dfTotal,dfTotal.ACCOUNT==dfBufferOrig.ACCOUNTPK).drop(dfTotal.ACCOUNT).persist()

In [ ]:
dfAnVar.printSchema()

In [ ]:
dfAnVar.select('BALANCE').collect()

In [ ]:
total=dfAnVar.select('BALANCE').collect()

In [ ]:
cluster=dfAnVar.where(col('prediction')==4).select('BALANCE').collect()

In [ ]:
stats.f_oneway(total,cluster)

In [ ]:
dfTrainRes.printSchema()

#### Products

In [ ]:
#dfAnalysis=spark.read.parquet("/analytics/bc1_clustering/tables/dfAnalysisBin")
dfAnalysis=spark.read.parquet("/analytics/bc1_clustering/tables/dfAnalysis11Dic")

In [ ]:
dfAnalysis.printSchema()

In [ ]:
dfAnTmp1=dfAnalysis.join(dfTrainRes,dfTrainRes.ACCOUNT==dfAnalysis.ACCOUNTPK)

In [ ]:
dfAnTmp1.printSchema()

In [ ]:
dfAnTmp1.createOrReplaceTempView('tdfAnTmp1')

In [ ]:
dfSectorImportance=spark.sql("""SELECT * FROM (
    SELECT prediction, SECTOR, IMPORTANCE, 
    row_number() over (partition by prediction order by IMPORTANCE desc) as orden FROM 
    (
    SELECT prediction, SECTOR, AVGPRED/AVGTOTAL AS IMPORTANCE FROM 
    (SELECT PRF_DS_SETOR as SECTOR, SUM(PURCHASES)/COUNT(DISTINCT ACCOUNT) AS AVGTOTAL
    FROM tdfAnTmp1 GROUP BY PRF_DS_SETOR ORDER BY SECTOR) A,
    (SELECT prediction, PRF_DS_SETOR as SECTORPRED, SUM(PURCHASES)/COUNT(DISTINCT ACCOUNT) AS AVGPRED
    FROM tdfAnTmp1 GROUP BY prediction, PRF_DS_SETOR ORDER BY SECTORPRED) B
    WHERE SECTOR=SECTORPRED ORDER BY prediction, SECTOR) MAIN
    ORDER BY prediction, orden) FINAL
    WHERE orden=1""")
dfSectorImportance.printSchema()

In [ ]:
dfSectorImportance.toPandas()

#### Demographic

In [ ]:
#LECTURA DE TABLA PARQUET
dfPeople=spark.read.parquet("/analytics/bc1_clustering/tables/dfPeople")
dfPeople=dfPeople.withColumnRenamed("SERNO","PEOPLESERNO").withColumn('YOB',(dfPeople["DOB"]/10000).cast('integer'))
dfAccounts=spark.read.parquet("/analytics/bc1_clustering/tables/dfAccountsRefdate")
dfAccounts=dfAccounts.withColumnRenamed("SERNO","ACCOUNTSERNO").withColumnRenamed("PEOPLESERNO","PEOPLESERNOFK")

In [ ]:
dfDemographic=dfTrainRes.join(dfAccounts,dfAccounts.ACCOUNTSERNO==dfTrainRes.ACCOUNT) \
    .join(dfPeople,dfPeople.PEOPLESERNO==dfAccounts.PEOPLESERNOFK) \
    .select('ACCOUNTSERNO','prediction','PEOPLESERNO','SEX','MARITAL_STATUS','YOB').persist()

In [ ]:
dfDemographic.createOrReplaceTempView("tDemographic")

In [ ]:
dfAge=spark.sql("""SELECT prediction, round(AVG(YOB),0) as cuenta from tDemographic group by prediction order by prediction""")
dfAge.toPandas()

In [ ]:
dfSex=spark.sql("""SELECT A.*, B.total, round(cuenta/total,2) as porcentaje FROM 
    (SELECT prediction, SEX, COUNT(*) as cuenta from tDemographic group by prediction, SEX) A,
    (SELECT prediction, COUNT(*) as total from tDemographic group by prediction) B
    WHERE A.prediction=B.prediction order by prediction, SEX""")
dfSex.toPandas()

In [ ]:
dfMarital=spark.sql("""SELECT A.*, B.total, round(cuenta/total,2) as porcentaje FROM 
    (SELECT prediction, MARITAL_STATUS, COUNT(*) as cuenta from tDemographic group by prediction, MARITAL_STATUS) A,
    (SELECT prediction, COUNT(*) as total from tDemographic group by prediction) B
    WHERE A.prediction=B.prediction order by prediction, MARITAL_STATUS""")
dfMarital.toPandas()

### Model saving

In [ ]:
#Iter2 -> grupo, part, dinero, 6*IQR

In [29]:
model.save("/analytics/bc1_clustering/models/model04EneSubNWVal")

In [ ]:
sparkFeaturePipelineModel.save("/analytics/bc1_clustering/models/pipeline02EneGrpNWVal")

In [ ]:
transformer.save("/analytics/bc1_clustering/models/scaleFinancial02EneGrpNWVal")

In [ ]:
dfOrden.write.parquet("/analytics/bc1_clustering/tables/dfOrden02EneGrpNWVal")

In [ ]:
dfOrden=spark.read.parquet("/analytics/bc1_clustering/tables/dfOrden02EneGrpNWVal")

In [ ]:
transformer=ElementwiseProduct.load("/analytics/bc1_clustering/models/scaleFinancial02EneGrpNWVal")

In [ ]:
sparkFeaturePipelineModel=PipelineModel.load("/analytics/bc1_clustering/models/pipeline02EneGrpNWVal")

In [21]:
model=BisectingKMeansModel.load("/analytics/bc1_clustering/models/model04EneSubNWVal")